In [1]:
# all imorts
import carla
import random


In [2]:
# connect to Carla
client = carla.Client('localhost', 2000)
world = client.get_world()

# get a spawn point
spawn_points = world.get_map().get_spawn_points()

# get a vehicle from the library
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.filter('*vehicle.lincoln.mkz_2020*')

start_point = spawn_points[0]

# spawn a vehicle
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [3]:
# get the car's positon on the map
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471010, z=-0.004610), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [4]:
# initial spawn point is the same -just 0.6m hiher off the ground
print(start_point)

Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [5]:
# send vehicle off
vehicle.set_autopilot()

In [6]:
# get actual postion from the car movingvehicle
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-48.149189, y=15.404507, z=-0.004097), Rotation(pitch=-0.070255, yaw=-53.940308, roll=1.152731))


In [7]:
# now look at the map
town_map = world.get_map()

In [8]:
type(town_map)

carla.libcarla.Map

In [9]:
print(town_map)

Map(name=Carla/Maps/Town10HD_Opt)


In [10]:
roads =town_map.get_topology()

In [11]:
# topology is pairs of waypoints defining all roads - tuples of those
print(roads)

[(<carla.libcarla.Waypoint object at 0x000001B5F10AB990>, <carla.libcarla.Waypoint object at 0x000001B5F10AB930>), (<carla.libcarla.Waypoint object at 0x000001B5F10ABAB0>, <carla.libcarla.Waypoint object at 0x000001B5F10AB810>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB630>, <carla.libcarla.Waypoint object at 0x000001B5F10AB450>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB6F0>, <carla.libcarla.Waypoint object at 0x000001B5F10AB750>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB5D0>, <carla.libcarla.Waypoint object at 0x000001B5F10AB570>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB3F0>, <carla.libcarla.Waypoint object at 0x000001B5F10AB510>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB270>, <carla.libcarla.Waypoint object at 0x000001B5F10AB330>), (<carla.libcarla.Waypoint object at 0x000001B5F10AB2D0>, <carla.libcarla.Waypoint object at 0x000001B5EAC091B0>), (<carla.libcarla.Waypoint object at 0x000001B5EAC09330>, <carla.libcarla.Waypoint objec

In [12]:
# have a look ata a waypoint - itis transform wrapped as a waypoint
print(roads[0][0])

Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [13]:
len(roads)

200

In [14]:
# making a route
#from onw postion to another

In [15]:
# import some code coming with the sim
import sys
sys.path.append('Z:\CARLA_0.9.15\WindowsNoEditor\PythonAPI\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [16]:
# using the code to plan the route and then draw it in the simulator
town10hd_map = world.get_map()

sampling_resolution = 2

grp = GlobalRoutePlanner(town_map, sampling_resolution)

point_a = carla.Location(x=-64.644844, y=24.471010, z=-0.004847)
point_b = carla.Location(x=14.355174, y=-67.902527, z=-0.004591)

route = grp.trace_route(point_a, point_b) #there are other function can be use

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', 
                            draw_shadow = False, 
                            color = carla.Color(r=0 , g=0 , b=255), 
                            life_time = 120.0, persistent_lines= True)
    

In [17]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


In [18]:
# now we define 2 vehicles
lincoln_bp = world.get_blueprint_library().filter('*vehicle.lincoln.mkz_2020*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

#start first car in the already defined stard point
lincoln = world.try_spawn_actor(lincoln_bp[0], start_point)

In [19]:
# tweak spectator position to watch the show

spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectatoe_pos = carla.Transform(start_point.location + carla.Location(x=20, y=2, z=3),
                               carla.Rotation(yaw = start_point.rotation.yaw - 180))
spectator.set_transform(spectatoe_pos)

In [20]:
# drop the mini from sky

# Get blueprints and spawn points
blueprint_library = world.get_blueprint_library() # Make sure the blueprint is properly assigned
mini_bp = blueprint_library.find('vehicle.audi.a2')  # Mini Cooper blueprint

# Use a valid spawn point from the list, e.g., spawn_points[10]
if len(spawn_points) > 10:
    spawn_point = spawn_points[0]
    # Adjust the z-coordinate for ground level
    spawn_point.location.z = world.get_map().get_waypoint(spawn_point.location).transform.location.z
    
    # Apply an offset to the spawn point (e.g., move slightly above)
    offset_spawn_point = carla.Transform(
        spawn_point.location + carla.Location(x=-3, y=0, z=10),  # Adjust the Z value for offset
        spawn_point.rotation  # Maintain the same rotation as the spawn point
    )

    try:
        # Try to spawn the actor at the new position with the adjusted offset
        mini = world.try_spawn_actor(mini_bp, offset_spawn_point)
        if mini is None:
            print("Failed to spawn even with an offset.")
        else:
            print("Mini Cooper spawned successfully.")
            # Focus spectator on the spawned vehicle
            spectator = world.get_spectator()
            spawn_points = world.get_map().get_spawn_points()
            start_point = spawn_points[0]

            spectatoe_pos = carla.Transform(start_point.location + carla.Location(x=20, y=2, z=3),
                               carla.Rotation(yaw = start_point.rotation.yaw - 180))
            spectator.set_transform(spectatoe_pos)

    except Exception as e:
        print(f"Error during spawn: {e}")
else:
    print("Spawn point 10 does not exist.")



Mini Cooper spawned successfully.


In [21]:
import carla
import random
import time

# Connect to the CARLA client
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Get the vehicle blueprint library
blueprint_library = world.get_blueprint_library()

# List available vehicle blueprints
vehicle_blueprints = blueprint_library.filter('vehicle.*')
for blueprint in vehicle_blueprints:
    print(blueprint.id)  # This will print the available vehicle blueprints

# Update the vehicle_types list with available vehicles
vehicle_types = ['vehicle.tesla.model3', 'vehicle.audi.a2', 'vehicle.bmw.grandtourer']

# Get all the spawn points available in the world
spawn_points = world.get_map().get_spawn_points()

# Function to spawn traffic vehicles
def spawn_traffic(num_vehicles):
    vehicles = []  # To store spawned vehicle actors
    for _ in range(num_vehicles):
        # Pick a random vehicle type from the list
        vehicle_bp = random.choice(vehicle_types)

        # Pick a random spawn point from the available spawn points
        spawn_point = random.choice(spawn_points)

        
        # Ensure spawn point is valid
        vehicle = None
        for _ in range(10):  # Try up to 10 times to spawn at a valid location
            vehicle = world.try_spawn_actor(blueprint_library.find(vehicle_bp), spawn_point)
            if vehicle is not None:
                print(f"Spawned vehicle: {vehicle.type_id} at location: {spawn_point.location}")
                break
            else:
                # If spawn fails, try another spawn point
                spawn_point = random.choice(spawn_points)
                print(f"Spawn failed at {spawn_point.location}, trying again.")
        
        if vehicle is not None:
            # Enable autopilot so the vehicle will drive around
            vehicle.set_autopilot(True)
            # Add the vehicle to the list of vehicles
            vehicles.append(vehicle)
        else:
            print("Failed to spawn vehicle after multiple attempts.")
    
    return vehicles

# Function to start the traffic
def start_traffic():
    # Spawn 10 vehicles in the world (you can adjust the number as needed)
    traffic_vehicles = spawn_traffic(10)

    # Run traffic for 30 seconds (you can adjust this duration)
    time.sleep(30)

    # Destroy all vehicles after the traffic duration
    for vehicle in traffic_vehicles:
        vehicle.destroy()
    print("Traffic destroyed.")

# Run the traffic generation
start_traffic()

vehicle.audi.a2
vehicle.nissan.micra
vehicle.audi.tt
vehicle.mercedes.coupe_2020
vehicle.bmw.grandtourer
vehicle.harley-davidson.low_rider
vehicle.ford.ambulance
vehicle.micro.microlino
vehicle.carlamotors.firetruck
vehicle.carlamotors.carlacola
vehicle.carlamotors.european_hgv
vehicle.ford.mustang
vehicle.chevrolet.impala
vehicle.lincoln.mkz_2020
vehicle.citroen.c3
vehicle.dodge.charger_police
vehicle.nissan.patrol
vehicle.jeep.wrangler_rubicon
vehicle.mini.cooper_s
vehicle.mercedes.coupe
vehicle.dodge.charger_2020
vehicle.ford.crown
vehicle.seat.leon
vehicle.toyota.prius
vehicle.yamaha.yzf
vehicle.kawasaki.ninja
vehicle.bh.crossbike
vehicle.mitsubishi.fusorosa
vehicle.tesla.model3
vehicle.gazelle.omafiets
vehicle.tesla.cybertruck
vehicle.diamondback.century
vehicle.mercedes.sprinter
vehicle.audi.etron
vehicle.volkswagen.t2
vehicle.lincoln.mkz_2017
vehicle.dodge.charger_police_2020
vehicle.vespa.zx125
vehicle.mini.cooper_s_2021
vehicle.nissan.patrol_2021
vehicle.volkswagen.t2_2021
Spa

KeyboardInterrupt: 

In [ ]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


: 